# Transfer Learning Model Training with MobileNet for Casting Defect Detection use case 

## Dataset available here: https://www.kaggle.com/ravirajsinh45/real-life-industrial-dataset-of-casting-productm

In [ ]:
import tensorflow.keras
import tensorflow as tf
tf.compat.v1.enable_eager_execution(config=None, device_policy=None, execution_mode=None)
import numpy as np
from  tensorflow.keras.utils import multi_gpu_model 
from tensorflow.keras import models
from tensorflow.keras.preprocessing import image
from tensorflow.keras import applications,layers,models,optimizers

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
os.listdir("../Data")

In [ ]:
import zipfile
with zipfile.ZipFile("../Data/casting_data.zip", 'r') as zip_ref:
    zip_ref.extractall("../Data/castingData")

In [ ]:
mainpath='../Data/castingData/'

In [ ]:
path=mainpath+'train/'
path2=mainpath+'validation/'
img_height, img_width=224,224
batch_size=32

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Training Data Generator

In [ ]:
train_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(path,target_size=(img_height, img_width),
batch_size=batch_size,class_mode='categorical')

### Validation Data Generator

In [ ]:
val_datagen = ImageDataGenerator(rescale=1. / 255)

val_generator = val_datagen.flow_from_directory(path2,target_size=(img_height, img_width),
batch_size=batch_size,class_mode='categorical')

### Transfer Learning

In [ ]:
baseModel=applications.MobileNet(weights = "imagenet", include_top=False,input_shape = (img_width, img_height, 3))

In [ ]:
x = baseModel.output
x = layers.Flatten(name="flatten")(x)
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(2, activation='softmax')(x)

model = models.Model(inputs=baseModel.input, outputs=predictions)

In [ ]:
model.summary()

### Model Compilation

In [ ]:
# compile the model 
model.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adam(lr=0.0001), metrics=["accuracy"])

### Model Training

In [ ]:
model.fit_generator(train_generator,validation_data=val_generator,steps_per_epoch=train_generator.samples//batch_size,
                    validation_steps=val_generator.samples//batch_size,epochs=4)

### Model conversion to ONNX

In [ ]:
!pip install tf2onnx==1.9.3

In [ ]:
# convert to onnx model
tf_version = tf.__version__
tf.__version__ = "2.0.1"
import tf2onnx
temp_model_file = '../Model/castingDefectModelViaJNB.onnx'
onnx_model = tf2onnx.convert.from_keras(model, output_path=temp_model_file)
tf.__version__ = tf_version